<a href="https://colab.research.google.com/github/baker371/k8-data-visualization/blob/lwasampijja-baker/upwork-devs/lwasampijja-baker/Visualise_Zenhub_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#@title

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

%reload_ext google.colab.data_table

!cp "/content/drive/My Drive/models/issues.py" .


from issues import *

repos = Repos.get_repos()

issues = Issues(repos)

issues.show_tabular_report_by_repo()

Mounted at /content/drive


pipeline,,Backlog,In Progress,New Issues,Peer Review
repo,,,,,
Open-Source,24,0,2,3,1
aws-jmeter-test-engine-v1,30,0,0,0,0
chrome-extension,7,0,4,4,4
glasswall-desktop,1,0,0,0,0
icap-poc-load-balancer,7,0,0,0,0
k8-data-visualization,23,1,5,0,1
k8-glasswall-rebuild,24,0,1,4,1
k8-performance-use-cases,17,1,1,2,2
k8-proxy-desktop,12,2,5,11,0


In [7]:
#@title
issues.show_tabular_report_by_user()

pipeline,,Backlog,In Progress,New Issues,Peer Review
assignee.login,,,,,
Abby595,0,0,0,1,0
DGBREW,2,0,0,1,0
DinisCruz,1,0,0,0,0
GiuseMSD,1,0,0,2,0
Infospectives,0,0,0,3,0
IslamHeggy,1,0,0,0,1
JOSHUAJEBARAJ,2,0,0,0,0
JameelNabbo,2,0,0,4,0
M-Ayman,0,0,0,0,1
